# Exploiting CVE-2019-1388
Detects an explotation attempt in which the UAC consent dialogue is used to invoke an Internet Explorer process running as LOCAL_SYSTEM

## Rule Content
```
- title: Exploiting CVE-2019-1388
  id: 02e0b2ea-a597-428e-b04a-af6a1a403e5c
  status: experimental
  description: Detects an explotation attempt in which the UAC consent dialogue is
    used to invoke an Internet Explorer process running as LOCAL_SYSTEM
  references:
  - https://portal.msrc.microsoft.com/en-US/security-guidance/advisory/CVE-2019-1388
  - https://www.zerodayinitiative.com/blog/2019/11/19/thanksgiving-treat-easy-as-pie-windows-7-secure-desktop-escalation-of-privilege
  author: Florian Roth
  date: 2019/11/20
  tags:
  - attack.privilege_escalation
  - attack.t1068
  logsource:
    category: process_creation
    product: windows
    service: null
  detection:
    selection:
      ParentImage: '*\consent.exe'
      Image: '*\iexplore.exe'
      CommandLine: '* http*'
    rights1:
      IntegrityLevel: System
    rights2:
      User: NT AUTHORITY\SYSTEM
    condition: selection and ( rights1 or rights2 )
  falsepositives:
  - Unknown
  level: critical

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='((process_parent_path.keyword:*\\consent.exe AND process_path.keyword:*\\iexplore.exe AND process_command_line.keyword:*\ http*) AND (IntegrityLevel:"System" OR user_account:"NT\ AUTHORITY\\SYSTEM"))')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()